In [1]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['GORDON_REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from enum import Enum

class ProposalType(Enum):
    GLOBAL = 'global'
    LOCAL = 'local'
    FREEFORM = 'freeform'
    
class PolygonType(Enum):
    CLOSED = 'closed'
    OPEN = 'open'
    TEXTURE = 'textured'
    TEXTURE_WITH_CONTOUR = 'texture with contour'
    DIRECTION = 'directionality'
    
from matplotlib.path import Path
%matplotlib inline

In [3]:
dms = dict([(sc, DataManager(stack='MD593', section=sc, segm_params_id='tSLIC200', load_mask=False)) 
            for sc in range(60, 151)])

In [4]:
for dm in dms.itervalues():
    dm.load_multiple_results(['texHist', 'spCentroids', 'edgeMidpoints', 'edgeEndpoints'])

In [5]:
label_examples = defaultdict(list) # cluster, dedges, sig, stack, sec, proposal_type, labeling username, timestamp

for sec, dm in dms.iteritems():
    
    results = dm.load_proposal_review_result('yuncong', 'latest', 'all')
    
    if len(results) == 0:
        continue
    
    for usr, ts, suf, res in results:

        global_proposals = dm.load_pipeline_result('proposals')
        local_proposals = dm.load_pipeline_result('allSeedClusterScoreDedgeTuples')

        if suf == 'globalProposalReviewResult':
            for i, (label, accepted) in enumerate(zip(*res)): 
                if accepted == 1:
                    cl, de, sig = global_proposals[i]
                    label_examples[label].append((cl, de, sig, dm.stack, dm.slice_ind, ProposalType.GLOBAL, usr, ts))

        if suf == 'localProposalReviewResult':
            for i, (label, accepted) in enumerate(zip(*res)): 
                if accepted == 1:
                    se, cl, sig, de = local_proposals[i]
                    label_examples[label].append((cl,de,sig, dm.stack, dm.slice_ind, ProposalType.LOCAL, usr, ts))

        if suf == 'freeformProposals':


            for label, typed_polygons in res.iteritems(): 
                for poly_type, vertices in typed_polygons:
                    pp = Path(vertices)
                    cl = np.where([pp.contains_point(s) for s in dm.sp_centroids[:, ::-1]])[0]
                    de = dm.find_boundary_dedges_ordered(cl)
                    sig = dm.compute_cluster_score(cl, method='rc-mean')[0]
                    
                    label_examples[label].append((cl, de, sig, dm.stack, dm.slice_ind, ProposalType.FREEFORM, usr, ts))

label_examples.default_factory = None

In [6]:
label_examples.keys()

['spinal nerve',
 'pontine',
 'facial motor nerve',
 'lateral reticular nucleus',
 'nucleus ambuggus',
 'facial motor nucleus']

In [7]:
label_texture = {}

for label, proposals in label_examples.iteritems():
    w = []
    for prop in proposals:
        cluster, dedges, sig, stack, sec, proposal_type, username, timestamp = prop
        w.append(dms[sec].texton_hists[cluster].mean(axis=0))
        
    label_texture[label] = np.mean(w, axis=0)

In [8]:
def fit_ellipse_to_points(pts):

    pts = np.array(list(pts) if isinstance(pts, set) else pts)

    c0 = pts.mean(axis=0)

    coords0 = pts - c0

    U,S,V = np.linalg.svd(np.dot(coords0.T, coords0)/coords0.shape[0])
    v1 = U[:,0]
    v2 = U[:,1]
    s1 = np.sqrt(S[0])
    s2 = np.sqrt(S[1])

    return v1, v2, s1, s2, c0

In [ ]:
# label_position = {}

# for label, proposals in label_examples.iteritems():
#     all_pts = []
#     for prop in proposals:
#         cluster, dedges, sig, stack, sec, proposal_type, username, timestamp = prop
#         pts = np.vstack([ dms[sec].sp_coords[sp][:, ::-1] for sp in cluster])
#         all_pts.append(pts)
        
#     all_pts = np.vstack(all_pts)
    
#     ell = fit_ellipse_to_points(all_pts)
    
#     label_position[label] = ell

In [9]:
label_position = defaultdict(lambda: {})

for label, proposals in label_examples.iteritems():

    for prop in proposals:
        cluster, dedges, sig, stack, sec, proposal_type, username, timestamp = prop
        
        dms[sec].load_multiple_results(['spCoords'])
        
        pts = np.vstack([ dms[sec].sp_coords[sp][:, ::-1] for sp in cluster])
        ell = fit_ellipse_to_points(pts)
        label_position[label][sec] = ell
        
label_position.default_factory=None

In [10]:
label_position['pontine']

{135: (array([-0.41929907, -0.90784816]),
  array([-0.90784816,  0.41929907]),
  309.44362804375578,
  142.4915779486322,
  array([ 2010.68032119,  5401.04844145])),
 136: (array([-0.43456888, -0.9006386 ]),
  array([-0.9006386 ,  0.43456888]),
  282.16509208085409,
  114.96831207450401,
  array([ 1972.24087338,  5346.35578873])),
 137: (array([ 0.40027477,  0.91639517]),
  array([ 0.91639517, -0.40027477]),
  254.71360173433311,
  117.40960090041217,
  array([ 1947.6710254 ,  5261.16708957]))}

In [11]:
from shapely.geometry import Polygon

In [12]:
def vertices_from_dedges(sec, dedges):

    vertices = []
    for de_ind, de in enumerate(dedges):
        midpt = dms[sec].edge_midpoints[frozenset(de)]
        endpts = dms[sec].edge_endpoints[frozenset(de)]
        endpts_next_dedge = dms[sec].edge_endpoints[frozenset(dedges[(de_ind+1)%len(dedges)])]

        dij = cdist([endpts[0], endpts[-1]], [endpts_next_dedge[0], endpts_next_dedge[-1]])
        i,j = np.unravel_index(np.argmin(dij), (2,2))
        if i == 0:
            vertices += [endpts[-1], midpt, endpts[0]]
        else:
            vertices += [endpts[0], midpt, endpts[-1]]
        
    return vertices


In [33]:
label_polygon = defaultdict(lambda: {})

for label, proposals in label_examples.iteritems():

    for prop in proposals:
        cluster, dedges, sig, stack, sec, proposal_type, username, timestamp = prop
        vs = vertices_from_dedges(sec, dedges)
        polygon = Polygon(vs)
        label_polygon[label][sec] = polygon
        
#         pts = np.vstack([ dms[sec].sp_coords[sp][:, ::-1] for sp in cluster])
#         label_coords[label][sec] = pts

label_polygon.default_factory=None

In [ ]:
atlas = img_as_ubyte(dms[139].image_rgb_jpg)
for label, (v1,v2,s1,s2,c0) in label_position.iteritems():
    angle = np.rad2deg(np.arctan2(v1[1], v1[0]))
    cv2.ellipse(atlas, tuple(c0.astype(np.int)), (int(2*1.5*s1), int(2*1.5*s2)), int(angle), 0, 360, (0,255,0), 10)
display(atlas)

In [35]:
pickle.dump(label_examples, open(os.environ['GORDON_RESULT_DIR']+'/database/label_examples.pkl', 'w'))
pickle.dump(label_position, open(os.environ['GORDON_RESULT_DIR']+'/database/label_position.pkl', 'w'))
pickle.dump(label_polygon, open(os.environ['GORDON_RESULT_DIR']+'/database/label_polygon.pkl', 'w'))
pickle.dump(label_texture, open(os.environ['GORDON_RESULT_DIR']+'/database/label_texture.pkl', 'w'))